In [16]:
import mlflow
import pandas as pd

import mlflow
import pandas as pd

def generate_recommendations_table(experiment_ids, aggregation_function="common_features", note="sizes_acts", group_type="sim"):
    all_rows = []

    for exp_id in experiment_ids:
        runs = mlflow.search_runs(
            experiment_ids=[exp_id],
            output_format="list"
        )
        for run in runs:
            if run.data.params.get("note") != note or run.data.params.get("SAE_fusion_strategy") != aggregation_function or run.data.params.get("group_type") != group_type:
                continue

            dataset = run.data.params.get("dataset", f"Exp-{exp_id}")
            dim = int(run.data.params.get("embedding_dim", 0))
            topk = int(run.data.params.get("top_k", 0))

            row_key = (dim, topk)
            metrics = {
                (dataset, "G/mean"): run.data.metrics.get("CommonItemsNDCG20/mean"),
                (dataset, "U/mean"): run.data.metrics.get("NDCG20/mean"),
                (dataset, "U/min"): run.data.metrics.get("NDCG20/min"),
                (dataset, "Pop"): run.data.metrics.get("Popularity/mean"),
            }

            all_rows.append((row_key, metrics))

    # Build DataFrame from records
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Dimensions", "TopK"]

    # Sort and reindex columns by dataset then metric
    df = df.sort_index(axis=1, level=[0, 1]).sort_values(
        by=["Dimensions", "TopK"]
    )
    
    # Custom column sorting
    dataset_order = ["MovieLens", "LastFM1k"]
    metric_order = ["G/mean", "U/min", "U/mean", "Pop"]

    cols = df.columns
    cols = sorted(
        cols,
        key=lambda x: (
            dataset_order.index(x[0]) if x[0] in dataset_order else len(dataset_order),
            metric_order.index(x[1]) if x[1] in metric_order else len(metric_order)
        )
    )
    
    df = df[cols]

    return df.reset_index()

def generate_sae_table(experiment_ids, note=None):
    all_rows = []

    for exp_id in experiment_ids:
        runs = mlflow.search_runs(
            experiment_ids=[exp_id],
            output_format="list"
        )
        for run in runs:
            if run.data.params.get("note") != note:
                continue

            dataset = run.data.params.get("dataset", f"Exp-{exp_id}")
            dim = int(run.data.params.get("embedding_dim", 0))
            base_factors = float(run.data.params.get("base_factors", 0.0))
            topk = int(run.data.params.get("top_k", 0))

            row_key = (dim, topk)
            metrics = {
                (dataset, "CS"): run.data.metrics.get("CosineSim/test"),
                (dataset, "Deg"): run.data.metrics.get("NDCG20_Degradation/test"),
                (dataset, "Deads"): run.data.metrics.get("DeadNeurons/test")
            }

            all_rows.append((row_key, metrics))

    # Build DataFrame from records
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Dimensions", "TopK"]

    # Sort and reindex columns by dataset then metric
    df = df.sort_index(axis=1, level=[0, 1]).sort_values(
        by=["Dimensions", "TopK"]
    )
    
    # Now, sort the columns manually
    dataset_order = ["MovieLens", "LastFM1k"]
    metric_order = ["CS", "Deg", "Deads"]

    # Build the new ordered columns
    cols = df.columns
    cols = sorted(
        cols,
        key=lambda x: (
            dataset_order.index(x[0]) if x[0] in dataset_order else len(dataset_order),
            metric_order.index(x[1]) if x[1] in metric_order else len(metric_order)
        )
    )

    df = df[cols]

    return df.reset_index()

In [17]:
import mlflow
import pandas as pd

def generate_common_features_table(experiment_ids, note="sizes_acts"):
    all_rows = []

    for exp_id in experiment_ids:
        runs = mlflow.search_runs(
            experiment_ids=[exp_id],
            filter_string=f"params.note = '{note}'",
            output_format="list"
        )

        for run in runs:
            params = run.data.params
            metrics = run.data.metrics
            
            if params.get("note") != note:
                continue
            

            dataset = params.get("dataset", f"Exp-{exp_id}")
            group_type = params.get("group_type", "unknown").replace("random", "Rand").capitalize()
            dim = int(params.get("embedding_dim", 0))
            topk = int(params.get("top_k", 0))
            value = metrics.get("common_features/mean", None)

            if value is None:
                continue

            row_key = (dim, topk)
            all_rows.append((row_key, (dataset, group_type), value))

    # Create dictionary for DataFrame
    records = {}
    for row_key, col_key, value in all_rows:
        if row_key not in records:
            records[row_key] = {}
        records[row_key][col_key] = value

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Dimensions", "TopK"]

    # Reorder columns
    if not df.empty:
        datasets = ["MovieLens", "LastFM1k"]
        subcols = ["Sim", "Rand", "Outlier"]
        col_order = [(d, s) for d in datasets for s in subcols]
        df = df.reindex(columns=pd.MultiIndex.from_tuples(col_order)).sort_values(
            by=["Dimensions", "TopK"]
        )

    return df.reset_index()

# Sizes and selection of k for TopKSAE

## Table of reconstrictions metrics

- CS - Cosine Similarity of original and reconstructed embeddings
- Deads - Percentage of dead neurons in the sparse embedding
- Deg - Degradation of NDCG between ELSA model and ELSA + Autoencoder

In [18]:
sae_experiments = ['657713966175362303', '852893065079987597']

In [19]:
table = generate_sae_table(sae_experiments, note="sizes_L2").round(3)
table

Dimensions TopK MovieLens               LastFM1k              
                         CS    Deg  Deads       CS    Deg  Deads
0       1024   32     0.947 -0.012  0.000    0.903 -0.013  0.225
1       1024   64     0.970 -0.006  0.000    0.926 -0.007  0.083
2       1024  128     0.991 -0.002  0.000    0.949 -0.007  0.013
3       2048   32     0.943 -0.014  0.000    0.902 -0.002  0.532
4       2048   64     0.966 -0.007  0.000    0.925 -0.015  0.251
5       2048  128     0.987 -0.002  0.000    0.950 -0.019  0.097
6       4096   32     0.941 -0.015  0.007    0.898 -0.002  0.743
7       4096   64     0.961 -0.009  0.000    0.924  0.000  0.539
8       4096  128     0.978 -0.004  0.000    0.950 -0.008  0.303

In [20]:
def format_latex(df, highlight_max_cols=None, highlight_min_cols=None, round_digits=3):
    formatted_df = df.copy()
    highlight_max_cols = highlight_max_cols or []
    highlight_min_cols = highlight_min_cols or []

    for col in df.columns:
        col_values = df[col]

        if col in highlight_max_cols:
            top_two = col_values.nlargest(2).values

            def format_cell(val):
                if val == top_two[0]:
                    return f"\\textbf{{{val:.{round_digits}f}}}"
                elif val == top_two[1]:
                    return f"\\underline{{{val:.{round_digits}f}}}"
                else:
                    return f"{val:.{round_digits}f}"

        elif col in highlight_min_cols:
            bottom_two = col_values.nsmallest(2).values

            def format_cell(val):
                if val == bottom_two[0]:
                    return f"\\textbf{{{val:.{round_digits}f}}}"
                elif val == bottom_two[1]:
                    return f"\\underline{{{val:.{round_digits}f}}}"
                else:
                    return f"{val:.{round_digits}f}"
        else:
            def format_cell(val):
                return f"{val}"

        formatted_df[col] = col_values.apply(format_cell)

    return formatted_df

highlight_max_cols = [("MovieLens", "CS"), ("LastFM1k", "CS"), ("MovieLens", "Deg"), ("LastFM1k", "Deg")]
highlight_min_cols = [("MovieLens", "Deads"), ("LastFM1k", "Deads")]

formatted_table = format_latex(table, highlight_max_cols=highlight_max_cols, highlight_min_cols=highlight_min_cols)

formatted_table.to_latex(
    "sae_table.tex",
    index=False,
    float_format="%.3f",
    bold_rows=True,
    column_format="ll|rrr|rrr",
    escape=False,
    caption="Table of reconstruction metrics for different SAE configurations. 'CS' stands for the cosine similarity between original and reconstructed embeddings; 'Deg' indicates the degradation of NDCG@20 between ELSA performance and ELSA with SAE reconstruction.; 'Deads' represents the ratio of dead neurons to the number of dimensions",
    label="tab:sizes:reconstruction"
)

## Number of common activated neurons for different group types and datasets

- Sim - Similary groups
- Random - Random groups
- Outlier - groups with 1 outlier

In [21]:
experiments = ['228719589483846826','962723054918039068']
table = generate_common_features_table(experiments, note="sizes_L2_with_acts")
table

Dimensions TopK MovieLens                 LastFM1k                
                        Sim    Rand Outlier      Sim    Rand Outlier
0       1024   32     3.395   2.088   0.376    4.157   2.059   0.612
1       1024   64     8.307   5.867   2.568    9.200   4.616   1.748
2       1024  128    17.715  14.631  11.100   24.264  14.047   7.124
3       2048   32     2.748   1.707   0.226    3.002   1.181   0.164
4       2048   64     6.253   4.375   1.316    7.143   3.251   0.835
5       2048  128    16.270  13.027   7.960   18.680   9.837   3.955
6       4096   32     2.027   1.310   0.143    2.821   0.995   0.111
7       4096   64     3.833   2.560   0.495    6.058   2.575   0.537
8       4096  128     8.754   6.188   2.376   15.675   7.627   2.460

In [22]:

table.to_latex(
    "sae_table.tex",
    index=False,
    float_format="%.1f",
    bold_rows=True,
    column_format="ll|ccc|ccc",
    escape=False,
    caption="Table of mean activated dimension in the sparse embedding that are shared across all group members for similar (Sim), random (Rand) and outlier (Outlier) groups",
    label="tab:sizes:common-features"
)

## SAE group recommendation performance for Common Feature aggregation function and similar groups

- G/mean - averaged NDCG@20, where positive interactions are the ones that are shared by all group members
- Pop - Averaged popularity of recommended items, where popularity of item is defined as the number of users that interacted with it divided by the number of users that interacted with the most popular item in the dataset
- U/mean - averaged individual mean NDCG@20 across all group members
- U/min - averaged individual minimum NDCG@20 across all group members

In [23]:
experiment_ids = ['333391697323445885', '523100174176986081']

# nejdrive common_features
table = generate_recommendations_table(experiment_ids, aggregation_function="common_features", note="sizes_L2_with_acts")
table

Dimensions TopK MovieLens                                LastFM1k            \
                     G/mean     U/min    U/mean       Pop    G/mean     U/min   
0       1024   32  0.444326  0.399303  0.553612  0.434318  0.420743  0.483650   
1       1024   64  0.519551  0.460865  0.610138  0.436848  0.458571  0.523677   
2       1024  128  0.539815  0.479291  0.630126  0.429481  0.510365  0.573216   
3       2048   32  0.417945  0.380461  0.529717  0.415023  0.384962  0.451579   
4       2048   64  0.483419  0.432728  0.585238  0.416320  0.443005  0.508045   
5       2048  128  0.529148  0.472595  0.626071  0.427030  0.500906  0.563941   
6       4096   32  0.398497  0.358970  0.509931  0.421835  0.372015  0.434735   
7       4096   64  0.489560  0.430203  0.587641  0.461638  0.425205  0.492598   
8       4096  128  0.505884  0.451982  0.607665  0.438252  0.484354  0.545309   

                       
     U/mean       Pop  
0  0.672798  0.561290  
1  0.714344  0.565819  
2  0.756198  0.561879  
3  0.635217  0.511174  
4  0.697998  0.551301  
5  0.747092  0.559422  
6  0.613003  0.480080  
7  0.682281  0.537012  
8  0.735272  0.571606

In [24]:
highlight_max_cols = [('MovieLens', 'G/mean'), ('MovieLens', 'U/min'), ('MovieLens', 'U/mean'), ('LastFM1k', 'G/mean'), ('LastFM1k', 'U/min'), ('LastFM1k', 'U/mean')]
highlight_min_cols = [('MovieLens', 'Pop'), ('LastFM1k', 'Pop')]

formatted_table = format_latex(
    table,
    highlight_max_cols=highlight_max_cols,
    highlight_min_cols=highlight_min_cols,
    round_digits=3
)

formatted_table.to_latex(
    "sae_table.tex",
    index=False,
    float_format="%.3f",
    bold_rows=True,
    column_format="ll|cccc|cccc",
    escape=False,
    caption = (
        "Table of recommendation metrics for different SAE configurations using the ComF aggregation function. "
        "'G/mean' stands for the mean NDCG@20, where the ground-truth recommendations are those seen by all group members; "
        "'U/min' is the mean of the minimum NDCG@20 across all group members; "
        "'U/mean' is the mean of the mean NDCG@20 across all group members; "
        "'Pop' is the mean popularity of recommended items."
    ),
    label="tab:sizes:recommendation:common-features"
)

## SAE group recommendation performance for Average aggregation function and similar groups

In [25]:
table = generate_recommendations_table(experiment_ids, aggregation_function="average", note="sizes_L2_with_acts")
averaged_values = table.mean(axis=0, numeric_only=True)
table

Dimensions TopK MovieLens                                LastFM1k            \
                     G/mean     U/min    U/mean       Pop    G/mean     U/min   
0       1024   32  0.643217  0.555209  0.701424  0.503368  0.582981  0.627360   
1       1024   64  0.646444  0.557127  0.703499  0.502286  0.582438  0.627599   
2       1024  128  0.648176  0.559314  0.704366  0.500283  0.584092  0.631381   
3       2048   32  0.645140  0.555426  0.701851  0.504289  0.590933  0.636514   
4       2048   64  0.645654  0.556998  0.702476  0.500007  0.590046  0.635751   
5       2048  128  0.647071  0.558196  0.704067  0.498256  0.590192  0.636343   
6       4096   32  0.642675  0.555507  0.701667  0.505257  0.597986  0.642241   
7       4096   64  0.644548  0.555708  0.701959  0.501800  0.598697  0.643377   
8       4096  128  0.647022  0.557480  0.703284  0.496908  0.593075  0.638227   

                       
     U/mean       Pop  
0  0.802659  0.573396  
1  0.801508  0.566946  
2  0.802059  0.563743  
3  0.805501  0.572453  
4  0.804719  0.571222  
5  0.804851  0.566373  
6  0.808661  0.572172  
7  0.808978  0.570940  
8  0.806542  0.570906

In [26]:
highlight_max_cols = [('MovieLens', 'G/mean'), ('MovieLens', 'U/min'), ('MovieLens', 'U/mean'), ('LastFM1k', 'G/mean'), ('LastFM1k', 'U/min'), ('LastFM1k', 'U/mean')]
highlight_min_cols = [('MovieLens', 'Pop'), ('LastFM1k', 'Pop')]

formatted_table = format_latex(
    table,
    highlight_max_cols=highlight_max_cols,
    highlight_min_cols=highlight_min_cols,
    round_digits=3
)

formatted_table.to_latex(
    "sae_table.tex",
    index=False,
    float_format="%.3f",
    bold_rows=True,
    column_format="ll|cccc|cccc",
    escape=False,
    caption = (
        "Table of recommendation metrics for different SAE configurations using the AVG aggregation function. "
        "'G/mean' stands for the mean NDCG@20, where the ground-truth recommendations are those seen by all group members; "
        "'U/min' is the mean of the minimum NDCG@20 across all group members; "
        "'U/mean' is the mean of the mean NDCG@20 across all group members; "
        "'Pop' is the mean popularity of recommended items."
    ),
    label="tab:sizes:recommendation:average"
)

## SAE group recommendation performance for Average aggregation function and random groups

In [13]:
generate_recommendations_table(experiment_ids, aggregation_function="average", note="sizes_L2_with_acts", group_type="random").round(3)

KeyboardInterrupt: 

## SAE group recommendation performance for Average aggregation function and outlier groups

In [ ]:
generate_recommendations_table(experiment_ids, aggregation_function="average", note="sizes_L2_with_acts", group_type="outlier")

Dimensions TopK MovieLens                                LastFM1k            \
                     G/mean     U/min    U/mean       Pop    G/mean     U/min   
0       1024   32  0.543199  0.485509  0.664698  0.482425  0.412174  0.449873   
1       1024   64  0.553268  0.492001  0.668956  0.480733  0.396571  0.435829   
2       1024  128  0.547485  0.489734  0.665914  0.474984  0.399265  0.437662   
3       2048   32  0.548574  0.486560  0.666764  0.483544  0.419989  0.452384   
4       2048   64  0.543875  0.487055  0.664380  0.474957  0.413595  0.450101   
5       2048  128  0.544708  0.486799  0.664735  0.469228  0.411010  0.445927   
6       4096   32  0.548628  0.486502  0.666379  0.483634  0.426089  0.456553   
7       4096   64  0.546088  0.485083  0.666050  0.475795  0.425328  0.457026   
8       4096  128  0.538524  0.482721  0.662898  0.467253  0.415651  0.450732   

                       
     U/mean       Pop  
0  0.714683  0.590547  
1  0.707945  0.576327  
2  0.706608  0.571840  
3  0.717692  0.587283  
4  0.714825  0.584924  
5  0.712312  0.577713  
6  0.722299  0.590218  
7  0.722274  0.584895  
8  0.715320  0.582347

In [ ]:
generate_recommendations_table(experiment_ids, aggregation_function="common_features", note="sizes_L2_with_acts", group_type="random").round(3)

Dimensions TopK MovieLens                      LastFM1k                     
                     G/mean  U/min U/mean    Pop   G/mean  U/min U/mean    Pop
0       1024   32     0.344  0.305  0.455  0.413    0.257  0.305  0.469  0.451
1       1024   64     0.445  0.388  0.547  0.442    0.305  0.359  0.550  0.520
2       1024  128     0.449  0.399  0.564  0.428    0.367  0.432  0.630  0.562
3       2048   32     0.310  0.283  0.429  0.386    0.202  0.245  0.388  0.369
4       2048   64     0.377  0.340  0.497  0.406    0.271  0.324  0.488  0.454
5       2048  128     0.460  0.407  0.568  0.438    0.340  0.406  0.600  0.538
6       4096   32     0.284  0.258  0.398  0.378    0.166  0.203  0.326  0.311
7       4096   64     0.400  0.349  0.503  0.448    0.239  0.290  0.447  0.419
8       4096  128     0.427  0.379  0.540  0.441    0.307  0.369  0.555  0.515

In [ ]:
generate_recommendations_table(experiment_ids, aggregation_function="common_features", note="sizes_L2_with_acts", group_type="outlier").round(3)

Dimensions TopK MovieLens                                LastFM1k            \
                     G/mean     U/min    U/mean       Pop    G/mean     U/min   
0       1024   32  0.114328  0.124710  0.264850  0.252694  0.110027  0.145825   
1       1024   64  0.224290  0.227544  0.393442  0.298174  0.153592  0.209925   
2       1024  128  0.276656  0.271586  0.458787  0.315618  0.226828  0.299906   
3       2048   32  0.085057  0.097969  0.233700  0.235695  0.049284  0.069877   
4       2048   64  0.165536  0.175576  0.337156  0.272780  0.089495  0.123042   
5       2048  128  0.289976  0.288453  0.460936  0.333524  0.184752  0.249989   
6       4096   32  0.074900  0.088109  0.214947  0.232894  0.028275  0.046455   
7       4096   64  0.138915  0.142882  0.286388  0.276807  0.080371  0.111962   
8       4096  128  0.259782  0.254381  0.427079  0.342764  0.140842  0.196652   

                       
     U/mean       Pop  
0  0.279880  0.282219  
1  0.381393  0.376730  
2  0.509348  0.454876  
3  0.170201  0.172029  
4  0.256113  0.246598  
5  0.440543  0.399051  
6  0.135573  0.139730  
7  0.231774  0.226683  
8  0.359595  0.340327